# Occupancy Grid Mapping Implementation

## Overview
This exercise implements a 2D occupancy grid mapping algorithm for robotics applications. The implementation processes laser scanner data and known robot poses to create a probabilistic map of the environment, distinguishing between occupied, free, and unknown spaces.

## Requirements
- Python 3.x
- NumPy
- Matplotlib
- Bresenham algorithm implementation (provided in `bresenham.py`)

## File Structure
```
.
├── ex3.py              # Main implementation file
├── ranges.data         # Laser scanner measurements
├── poses.data          # Robot pose data
├── bresenham.py        # Bresenham line algorithm implementation
└── read_me.md           # This documentation
```

## Key Components

### Data Files
- `ranges.data`: Contains 2D laser range measurements recorded by the robot. Each line corresponds to range measurements taken at specific time instances.
- `poses.data`: Contains the known robot poses as it moves through the environment. Each line corresponds to the robot's pose at a specific time instance.

### Core Functions

#### Probability Conversions
- `prob2logodds(p)`: Converts probability to log odds representation
- `logodds2prob(l)`: Converts log odds back to probability

#### Sensor Model
- `inv_sensor_model(cell, endpoint, prob_occ, prob_free)`: Implements the inverse sensor model
  - Returns `prob_occ` (0.9) for cells where laser beams end
  - Returns `prob_free` (0.35) for cells along the beam path
  - Uses `prior` (0.5) for unobserved cells

#### Coordinate Transformations
- `world2map(pose, gridmap, map_res)`: Converts world coordinates to map coordinates
- `ranges2cells(r_ranges, w_pose, gridmap, map_res)`: Converts range measurements to map cells
- `poses2cells(w_pose, gridmap, map_res)`: Converts world poses to map cells

#### Main Mapping Function
- `grid_mapping_with_known_poses(ranges_raw, poses_raw, occ_gridmap, map_res, prob_occ, prob_free, prior)`: Implements the complete occupancy grid mapping algorithm

### Visualization
- `plot_gridmap(gridmap)`: Visualizes the occupancy grid map using matplotlib

## Usage

```python
import ex3
import numpy as np
import matplotlib.pyplot as plt

# Set parameters
map_size = 100
map_res = 0.25
prior = 0.50
prob_occ = 0.90
prob_free = 0.35

# Load data
ranges_raw = np.loadtxt("ranges.data", delimiter=',', dtype='float')
poses_raw = np.loadtxt("poses.data", delimiter=',', dtype='float')

# Initialize gridmap
occ_gridmap = ex3.init_gridmap(map_size, map_res) + prior

# Run mapping algorithm
final_map = ex3.grid_mapping_with_known_poses(
    ranges_raw, poses_raw, occ_gridmap, map_res, 
    prob_occ, prob_free, prior
)

# Visualize result
ex3.plot_gridmap(final_map)
plt.show()
```

## Parameters

- `map_size`: Size of the environment in meters
- `map_res`: Resolution of the grid map in meters/cell
- `prior`: Prior probability for unknown cells (default: 0.5)
- `prob_occ`: Probability for occupied cells (default: 0.9)
- `prob_free`: Probability for free cells (default: 0.35)

## Implementation Details

### Occupancy Update Process
1. Converts initial probabilities to log odds for numerical stability
2. For each pose and corresponding range measurement:
   - Converts range measurements to map coordinates
   - Identifies cells along each laser beam using Bresenham's algorithm
   - Updates log odds for each cell based on the inverse sensor model
3. Converts final log odds back to probabilities

### Coordinate Systems
- World coordinates: Robot's actual position in meters
- Map coordinates: Discretized grid cells
- Origin is at the center of the grid

## Notes
- The implementation assumes known robot poses
- Uses log odds for numerical stability in probability updates
- Employs Bresenham's line algorithm for ray tracing
- The map is centered around the origin

## Common Issues and Solutions

1. **Memory Usage**: For large environments or fine resolutions, consider:
   - Increasing map resolution (larger `map_res`)
   - Processing data in batches

2. **Performance**: To improve processing speed:
   - Vectorize operations where possible
   - Use numpy's optimized functions
   - Consider implementing ray tracing in C++ if needed

3. **Numerical Stability**: The implementation uses log odds to prevent numerical underflow/overflow in probability calculations

